## Working with image data in `Mosaic`

First, we'll download some data to explore. We're going to use the [CelebA dataset](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html), a dataset of over 200k portraits, each annotated for 40 different attributes (e.g. "smiling", "bangs", "mustache").
- Download time: ~3 minutes
- Download size: 3.1G

In [ ]:
import pandas as pd

In [ ]:
DATASET_DIR = "."

In [ ]:
from mosaic.contrib.celeba import download_celeba
download_celeba(DATASET_DIR)

In [ ]:
dataset[]